# Sparse Graph Algorithms
- Graph representation, Shortest paths, Connected components
- Real examples: Social networks, Route planning

In [1]:
import numpy as np
from scipy import sparse
from scipy.sparse import csgraph
import matplotlib.pyplot as plt
print('Sparse graph algorithms module loaded')

Sparse graph algorithms module loaded


## Graphs as Sparse Matrices

**Adjacency matrix**: A[i,j] = weight of edge i→j

**Properties**:
- **Undirected**: A = A.T (symmetric)
- **Unweighted**: A[i,j] ∈ {0, 1}
- **Weighted**: A[i,j] = edge weight

**Sparse advantage**: Real networks have sparse connections

In [2]:
# Build small graph
edges = [
    (0, 1, 4), (0, 2, 2),
    (1, 2, 1), (1, 3, 5),
    (2, 3, 8), (2, 4, 10),
    (3, 4, 2)
]

n_nodes = 5
rows, cols, weights = zip(*edges)

print(f'Graph with {n_nodes} nodes, {len(edges)} edges\n')

# Directed graph
graph_directed = sparse.csr_array((weights, (rows, cols)), shape=(n_nodes, n_nodes))

print('Directed adjacency matrix:')
print(graph_directed.toarray())
print()

# Undirected graph (symmetric)
graph_undirected = graph_directed + graph_directed.T
print('Undirected adjacency matrix:')
print(graph_undirected.toarray())

Graph with 5 nodes, 7 edges

Directed adjacency matrix:
[[ 0  4  2  0  0]
 [ 0  0  1  5  0]
 [ 0  0  0  8 10]
 [ 0  0  0  0  2]
 [ 0  0  0  0  0]]

Undirected adjacency matrix:
[[ 0  4  2  0  0]
 [ 4  0  1  5  0]
 [ 2  1  0  8 10]
 [ 0  5  8  0  2]
 [ 0  0 10  2  0]]


## Shortest Path Algorithms

**Functions**:
- `shortest_path()`: All-pairs or single-source
- `dijkstra()`: Single source (non-negative weights)
- `bellman_ford()`: Single source (allows negative)
- `floyd_warshall()`: All-pairs

**Returns**: Distance matrix and predecessors

In [3]:
# Shortest paths
print('Shortest Paths\n')

# All-pairs shortest paths
dist_matrix, predecessors = csgraph.shortest_path(graph_directed, 
                                                  return_predecessors=True,
                                                  directed=True)

print('Distance matrix (all-pairs):')
print(dist_matrix)
print()

# Path from node 0 to node 4
start, end = 0, 4
print(f'Shortest path from node {start} to node {end}:')
print(f'  Distance: {dist_matrix[start, end]}')

# Reconstruct path
path = []
current = end
while current != start:
    path.append(current)
    current = predecessors[start, current]
path.append(start)
path = path[::-1]
print(f'  Path: {path}')

Shortest Paths

Distance matrix (all-pairs):
[[ 0.  4.  2.  9. 11.]
 [inf  0.  1.  5.  7.]
 [inf inf  0.  8. 10.]
 [inf inf inf  0.  2.]
 [inf inf inf inf  0.]]

Shortest path from node 0 to node 4:
  Distance: 11.0
  Path: [0, np.int32(1), np.int32(3), 4]


## Connected Components

**Problem**: Find groups of connected nodes

**Function**: `connected_components()`
- Returns: n_components, labels
- Works on undirected graphs

**Use**: Community detection, clustering

In [4]:
# Graph with multiple components
n = 10
graph_multi = sparse.lil_array((n, n))

# Component 1: nodes 0-3
graph_multi[0, 1] = graph_multi[1, 0] = 1
graph_multi[1, 2] = graph_multi[2, 1] = 1
graph_multi[2, 3] = graph_multi[3, 2] = 1

# Component 2: nodes 4-6
graph_multi[4, 5] = graph_multi[5, 4] = 1
graph_multi[5, 6] = graph_multi[6, 5] = 1

# Component 3: nodes 7-9
graph_multi[7, 8] = graph_multi[8, 7] = 1
graph_multi[8, 9] = graph_multi[9, 8] = 1

graph_multi_csr = graph_multi.tocsr()

print('Connected Components\n')
n_components, labels = csgraph.connected_components(graph_multi_csr, 
                                                    directed=False)

print(f'Number of components: {n_components}\n')
for comp in range(n_components):
    nodes = np.where(labels == comp)[0]
    print(f'Component {comp}: nodes {nodes.tolist()}')

Connected Components

Number of components: 3

Component 0: nodes [0, 1, 2, 3]
Component 1: nodes [4, 5, 6]
Component 2: nodes [7, 8, 9]


## Real Example: Social Network Analysis

**Problem**: Analyze friendship network
**Tasks**: Find communities, shortest friendship path

**Real application**: LinkedIn 'degrees of separation'

In [5]:
# Social network
n_people = 500
avg_friends = 20

print('Social Network Analysis')
print(f'  People: {n_people:,}')
print(f'  Avg friends: {avg_friends}\n')

# Build friendship graph
np.random.seed(42)
rows, cols = [], []
for person in range(n_people):
    n_friends = np.random.poisson(avg_friends)
    friends = np.random.choice(n_people, size=min(n_friends, n_people-1), replace=False)
    friends = friends[friends != person]
    
    for friend in friends:
        rows.extend([person, friend])
        cols.extend([friend, person])

friendship = sparse.coo_array((np.ones(len(rows)), (rows, cols)), 
                              shape=(n_people, n_people))
friendship_csr = friendship.tocsr()
friendship_csr.data = np.ones(friendship_csr.nnz)  # Remove duplicates effect

print(f'Network stats:')
print(f'  Edges: {friendship_csr.nnz // 2:,}')
print(f'  Density: {friendship_csr.nnz/n_people**2*100:.3f}%\n')

# Connected components (friend circles)
n_components, labels = csgraph.connected_components(friendship_csr, directed=False)
print(f'Connected components: {n_components}')
if n_components > 1:
    for i in range(n_components):
        size = (labels == i).sum()
        print(f'  Component {i}: {size} people')
else:
    print('  All people connected!\n')

# Degrees of separation
person_a, person_b = 0, 100
dist_matrix = csgraph.shortest_path(friendship_csr, directed=False, indices=person_a)
separation = dist_matrix[person_b]

print(f'\nDegrees of separation:')
print(f'  Person {person_a} to Person {person_b}: {int(separation)} degrees')

Social Network Analysis
  People: 500
  Avg friends: 20

Network stats:
  Edges: 9,754
  Density: 7.803%

Connected components: 1
  All people connected!


Degrees of separation:
  Person 0 to Person 100: 2 degrees


## Minimum Spanning Tree

**Problem**: Connect all nodes with minimum total edge weight

**Algorithms**:
- `minimum_spanning_tree()`: Returns sparse matrix of MST

**Use**: Network design, clustering

In [6]:
# Weighted graph
n = 8
np.random.seed(42)
graph_weighted = sparse.random(n, n, density=0.3, format='csr')
graph_weighted.data = np.random.randint(1, 20, size=graph_weighted.nnz)
graph_weighted = graph_weighted + graph_weighted.T  # Symmetric

print(f'Minimum Spanning Tree')
print(f'  Nodes: {n}')
print(f'  Edges: {graph_weighted.nnz // 2}\n')

# Compute MST
mst = csgraph.minimum_spanning_tree(graph_weighted)

print(f'MST properties:')
print(f'  Edges in MST: {mst.nnz}')
print(f'  Total weight: {mst.sum():.0f}')
print(f'  Expected edges: {n - 1} (for connected graph)')

Minimum Spanning Tree
  Nodes: 8
  Edges: 16

MST properties:
  Edges in MST: 7
  Total weight: 29
  Expected edges: 7 (for connected graph)


## Real Example: City Route Planning

**Problem**: Find shortest route between cities
**Data**: Road network with distances

**Application**: GPS navigation, logistics

In [7]:
# Road network
cities = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
n_cities = len(cities)

# Roads (city1, city2, distance_km)
roads = [
    (0, 1, 50), (0, 2, 80), (1, 2, 30),
    (1, 3, 90), (2, 3, 60), (2, 4, 70),
    (3, 5, 40), (4, 5, 50), (4, 6, 100),
    (5, 6, 30), (5, 7, 80), (6, 7, 60)
]

print('City Route Planning')
print(f'  Cities: {n_cities}')
print(f'  Roads: {len(roads)}\n')

# Build road network (undirected)
rows, cols, dists = [], [], []
for i, j, d in roads:
    rows.extend([i, j])
    cols.extend([j, i])
    dists.extend([d, d])

road_network = sparse.csr_array((dists, (rows, cols)), shape=(n_cities, n_cities))

# Find shortest routes from city A
start_city = 0  # City A
distances = csgraph.dijkstra(road_network, indices=start_city, directed=False)

print(f'Shortest distances from {cities[start_city]}:')
for i, city in enumerate(cities):
    if i != start_city:
        print(f'  {cities[start_city]} → {city}: {distances[i]:.0f} km')

# Specific route
start, end = 0, 7  # A to H
dist, pred = csgraph.dijkstra(road_network, indices=start, return_predecessors=True)

# Reconstruct path
path = []
current = end
while current != -9999 and current != start:
    path.append(current)
    current = pred[current]
if current == start:
    path.append(start)
    path = path[::-1]
    print(f'\nRoute {cities[start]} → {cities[end]}:')
    print(f'  Path: {" → ".join([cities[i] for i in path])}')
    print(f'  Total distance: {dist[end]:.0f} km')

City Route Planning
  Cities: 8
  Roads: 12

Shortest distances from A:
  A → B: 50 km
  A → C: 80 km
  A → D: 140 km
  A → E: 150 km
  A → F: 180 km
  A → G: 210 km
  A → H: 260 km

Route A → H:
  Path: A → B → D → F → H
  Total distance: 260 km


## Summary

### Graph Functions:
```python
from scipy.sparse import csgraph

# Shortest paths
dist, pred = csgraph.shortest_path(graph, return_predecessors=True)
dist = csgraph.dijkstra(graph, indices=source)

# Connected components
n_comp, labels = csgraph.connected_components(graph)

# Minimum spanning tree
mst = csgraph.minimum_spanning_tree(graph)

# Breadth-first search
bfs_tree = csgraph.breadth_first_tree(graph, source, directed=False)

# Depth-first search
dfs_tree = csgraph.depth_first_tree(graph, source, directed=False)
```

### Graph Representations:
- **Adjacency matrix**: A[i,j] = edge weight
- **Sparse CSR/CSC**: Efficient for real networks
- **Directed vs undirected**: A.T for symmetric

### Applications:

**Social Networks**:
- Degrees of separation
- Community detection
- Influence propagation

**Transportation**:
- Route planning (GPS)
- Network optimization
- Traffic flow

**Web**:
- PageRank (eigenvector)
- Link analysis
- Crawling paths

**Biology**:
- Protein interactions
- Gene networks
- Phylogenetic trees

### Performance:
- **Dijkstra**: O((V+E) log V) with sparse
- **Floyd-Warshall**: O(V³) - dense only
- **BFS/DFS**: O(V+E)
- **MST**: O(E log V)